In [0]:
# first we import liberires
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, desc, month, year, to_date

In [0]:
# now we load the data
file_path = "/Workspace/Users/deepakpvinodsharma@gmail.com/THE_LEARNING_BRICK/Databricks-begining-to-end/Sample - Superstore.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)

In [0]:
# now we clean the column names
for col in df.columns:
    df = df.withColumnRenamed(col, col.strip().replace(" ", "_"))

In [0]:
# now we create dataframe as temperary view
df.createOrReplaceTempView("sales")

In [0]:
# now we create total sales category 
query1 = spark.sql(""" SELECT category, ROUND(SUM("Sales"), 2), AS Total_sales
                   From samples
                   Group BY category
                   ORDER BY Total_sales DESC
                   """).query1.show()

In [0]:
# monthly sales trend
from pyspark.sql.functions import month, year, to_date
df_with_month = df.withColumn("Month", month("Order_Date")).withColumn("Year", year("Order_Date"))
df_with_month.createOrReplaceTempView("sales_by_month")

query2 = spark.sql(""" SELECT Year, Month, Round(SUM("Sales"), 2) AS Total_Sales 
                       FROM sales_by_month
                       GROUP BY Year, month
                       ORDER BY Year, month
                   """)
query2.show()

In [0]:
# customer names
query3 = spark.sql(""" SELECT Customer_Name, Round(SUM(Sales), 2), AS Total_Sales
                   FROM sales
                   GROUP BY Customer_Name
                   ORDER BY Total_Sales DESC
                   LIMIT 10

                   """)
query3.show()

In [0]:
# Maximum loss
query4 = spark.sql(""" SELECT Region, Round(SUM(Sales), 2), AS Total_Sales
                   FROM sales
                     GROUP BY Region
                     ORDER BY Total_Sales ASC
                     LIMIT 1
                     """)
query4.show()

In [0]:
query1.write.mode("overwrite").save("/Workspace/Users/deepakpvinodsharma@gmail.com/THE_LEARNING_BRICK/Databricks-begining-to-end/Sample - Superstore.csv")